In [1]:
import sys
import requests
sys.path.append('../../utils/')
import biotite_utils
import dataset_utils


In [2]:
def get_entity_id(pdb_id, chain_id):
    successful_request = False
    while not successful_request:
        try:
            pdb_info = requests.get(f"https://www.ebi.ac.uk/pdbe/api/pdb/entry/molecules/{pdb_id}").json()
            successful_request = True
        except:
            successful_request = False
    
    for entity in pdb_info[pdb_id]:
        if chain_id in entity['in_chains']:
            return entity['entity_id']
    return None

def get_conservation_score(pdb_id, chain_id):
    entity_id = get_entity_id(pdb_id, chain_id)
    successful_request = False
    while not successful_request:
        try:
            return_val = requests.get(f"https://www.ebi.ac.uk/pdbe/graph-api/pdb/sequence_conservation/{pdb_id}/{entity_id}").json()['data']
            successful_request = True
        except:
            successful_request = False

    return return_val
a = get_conservation_score('1a4u', 'B')


In [3]:
import biotite.structure.io.pdbx as pdbx
import biotite.structure as struc
CIF_FILES_PATH = '/home/vit/Projects/deeplife-project/data/cif_files'

# sometimes one AUTH chain can be described by two LABEL chains
def get_label_chain_id(pdb_id, auth_chain):
    mmcif_file = pdbx.CIFFile.read(f'{CIF_FILES_PATH}/{pdb_id}.cif')
    whole_structure = pdbx.get_structure(mmcif_file, model=1, include_bonds=True, use_author_fields=False)
    label_chains = struc.get_chains(whole_structure)
    whole_structure = pdbx.get_structure(mmcif_file, model=1, include_bonds=True)
    auth_chains = struc.get_chains(whole_structure)

    final = []
    for candidate_label_chain, candidate_auth_chain in zip(label_chains, auth_chains):
        if auth_chain == candidate_auth_chain:
            final.append(candidate_label_chain)
    return final


get_label_chain_id('2e1c', 'A')

['C', 'F']

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

DATASET= 'cryptobench-dataset'
FLUCTUATION_PATH = f'/home/vit/Projects/flexibility-analysis/data/features/fluctuation/{DATASET}/fluctuation'
CIF_FILES_PATH = '/home/vit/Projects/deeplife-project/data/cif_files'
OUTPUT_PATH = f'/home/vit/Projects/flexibility-analysis/data/features/conservation/{DATASET}'

train_set = dataset_utils.load_train_set(f'../../../datasets/{DATASET}')

for filename in os.listdir(FLUCTUATION_PATH):
    id = filename.split('.')[0]
    
    if f'{id}.npy' in os.listdir(f'{OUTPUT_PATH}'):
        continue
    if id == '8j1kA':
        continue

    print(f'Processing {id} ...')
    pdb_id = id[:4]
    chain_id = id[4:]
    label_chains = get_label_chain_id(pdb_id, chain_id)

    protein = biotite_utils.load_structure(pdb_id, use_author_fields=False)
    # some errors with non-standard residue types
    protein_backbone = protein[(protein.atom_name == "CA") 
                       & (protein.element == "C") 
                       & (np.isin(protein.chain_id, label_chains)) 
                       & (
                             (protein.res_name == 'ALA')
                           | (protein.res_name == 'ARG')
                           | (protein.res_name == 'ASN')
                           | (protein.res_name == 'ASP')
                           | (protein.res_name == 'CYS')
                           | (protein.res_name == 'GLN')
                           | (protein.res_name == 'GLU')
                           | (protein.res_name == 'GLY')
                           | (protein.res_name == 'HIS')
                           | (protein.res_name == 'ILE')
                           | (protein.res_name == 'LEU')
                           | (protein.res_name == 'LYS')
                           | (protein.res_name == 'MET')
                           | (protein.res_name == 'PHE')
                           | (protein.res_name == 'PRO')
                           | (protein.res_name == 'SER')
                           | (protein.res_name == 'THR')
                           | (protein.res_name == 'TRP')
                           | (protein.res_name == 'TYR')
                           | (protein.res_name == 'VAL'))]
    
    if len(protein_backbone) == 0: continue

    data = get_conservation_score(pdb_id, chain_id)

    conservation = []

    missing_conservation = 0
    for idx, residue in enumerate(protein_backbone):
        label_res_id = residue.res_id
        if label_res_id in data['index']:
            conservation.append(data['conservation_score'][data['index'].index(label_res_id)])
        else:
            conservation.append(0)
            missing_conservation += 1
    
    fluctuation_len = len(np.load(f'{FLUCTUATION_PATH}/{id}.npy'))
    assert len(protein_backbone) == len(conservation), f"{id}, {len(protein_backbone)} {len(conservation)}"
    
    if fluctuation_len != len(conservation): 
        print(f'{id}, {fluctuation_len} {len(conservation)}')
        continue

    # print(f'\t{id}: MISSING CONSERVATION - {missing_conservation}; TOTAL - {len(protein_backbone)}')
    # print('\tSaving to file ...') 
    np.save(f'{OUTPUT_PATH}/{id}.npy', np.array(conservation))

    def generate_fig(fluctuation, indices, trim_outliers=False):
        if trim_outliers:
            quantile_95 = np.quantile(fluctuation, 0.95)
            fluctuation = np.where(fluctuation < quantile_95, fluctuation, quantile_95)

        x = 0.5 + np.arange(len(fluctuation))

        a = plt.bar(x, fluctuation, width=1, color="y", linewidth=0.7)
        plt.title(f"{id}")
        plt.ylim((0, np.max(fluctuation)))
        plt.xlim((0, len(fluctuation)))

        for item in indices:
            assert item < len(a), f'{item}, {id}'
            a[item].set_color('r')

        # plt.savefig(f'{OUTPUT_PATH}/{id}.png', dpi=100)
        plt.show()

    # generate_fig(msqf, indices)


Processing 2yx7A ...
2yx7A, 151 150
Processing 6lgyA ...
6lgyA, 312 311
Processing 1k1xB ...
Processing 3iunA ...
Processing 7yx7A ...
Processing 6f4mA ...
Processing 6pntA ...
Processing 1fwkC ...
Processing 1p74A ...
Processing 3bifA ...
Processing 8h0nB ...
Processing 1pfzC ...
Processing 3pgsB ...
Processing 4ei8A ...
Processing 7jrbA ...
Processing 6up2A ...
Processing 1xqzA ...
Processing 3f0fB ...
Processing 4omgA ...
Processing 1g24C ...
Processing 6czpB ...
Processing 6aqeB ...
Processing 6j35B ...
Processing 8s32A ...
Processing 5lqjC ...
Processing 5ojxA ...
Processing 2eplX ...
Processing 2xtpA ...
Processing 7ymfA ...
Processing 1bfnA ...
Processing 6ialF ...
Processing 7moiB ...
Processing 2jfyB ...
Processing 2jfpA ...
Processing 4nfcA ...
Processing 3h11B ...
Processing 7djnA ...
Processing 4j2pA ...
Processing 2beiB ...
Processing 3iecB ...
Processing 1mwrA ...
Processing 5l02B ...
Processing 5z6cA ...
Processing 6iu0B ...
Processing 6ltfA ...
Processing 1sjsA ...
Proc

# Total number of structures

We calculated conservation only for 781 structures, as we considered only the single chain structures from the train set